In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

CSV_PATH = "/kaggle/input/dataset/observation.csv"        # change the Path to the dataset CSV file
SAVE_FULL = "/kaggle/working/traffic_transformer_model_2.pth"   # chenge the Path to the saved full model
INPUT_LEN = 72
BATCH_SIZE = 128
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TARGET_PLACE = "Alipore"     # change the target location as needed dynamically

CONGESTION_THRESHOLDS = {
    'LOW': 200 ,   
    'MEDIUM': 300 
  
}

def determine_congestion_level(mean_count, thresholds):
    """Determines the congestion level based on mean car count and predefined thresholds."""
    if mean_count <= thresholds['LOW']:
        return "Low "
    elif mean_count <= thresholds['MEDIUM']:
        return "Medium "
    else:
        return "High "

class HybridLoss(nn.Module):
    """alpha * L1 + (1-alpha) * SmoothL1"""
    def __init__(self, alpha=0.6, beta=0.5):
        super().__init__()
        self.l1 = nn.L1Loss()
        self.smooth = nn.SmoothL1Loss(beta=beta)
        self.alpha = alpha
    def forward(self, pred, target):
        return self.alpha * self.l1(pred, target) + (1 - self.alpha) * self.smooth(pred, target)

class TransformerForecaster(nn.Module):
    def __init__(self, input_size, hidden_size=256, num_heads=8, dropout=0.1, seq_len=INPUT_LEN):
        super().__init__()
        if input_size % num_heads != 0:
        self.pos_embedding = nn.Parameter(torch.randn(1, seq_len, input_size) * 0.01)
        self.layer_norm = nn.LayerNorm(input_size)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_size,
            nhead=num_heads,
            dim_feedforward=hidden_size,
            dropout=dropout,
            batch_first=True,
            activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        self.fc = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, 1)
        )
    def forward(self, x):
        x = x + self.pos_embedding[:, :x.size(1), :]
        x = self.layer_norm(x)
        x = self.transformer(x)
        x = x[:, -1, :] 
        return self.fc(x).squeeze(-1)

class TrafficDataset(Dataset):
    def __init__(self, data, input_len=INPUT_LEN, feature_dim=None):
        self.X, self.y = [], []
        for i in range(len(data) - input_len):
            self.X.append(data[i:i+input_len, :feature_dim])
            self.y.append(data[i+input_len, -1])
        self.X = torch.tensor(np.array(self.X), dtype=torch.float32)
        self.y = torch.tensor(np.array(self.y), dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

print("Loading and reprocessing data to re-fit scalers...")
try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    print(f"\n ERROR: File not found at {CSV_PATH}. Please fix the path and try again.")
    df = None 

if df is not None:
    df = df.drop(columns=["Datetime"], errors="ignore")

    def extract_hour(timeslot):
        if pd.isna(timeslot): return None
        try:
            start_time = timeslot.split('-')[0].strip()
            start_time = re.sub(r'(?i)(AM|PM)', r' \1', start_time).strip().upper()
            return pd.to_datetime(start_time, format='%I:%M %p').hour
        except:
            try: return pd.to_datetime(start_time, format='%H:%M').hour
            except: return None

    df['StartHour'] = df['TimeSlot'].apply(extract_hour)
    df.dropna(subset=['StartHour'], inplace=True)
    df['StartHour'] = df['StartHour'].astype(int)

    le_to = LabelEncoder()
    df['To_encoded'] = le_to.fit_transform(df['To'])

    df['DayOfYear'] = (df['Month'] - 1) * 30 + df['Day']
    df['Day_sin'] = np.sin(2 * np.pi * df['DayOfYear'] / 365)
    df['Day_cos'] = np.cos(2 * np.pi * df['DayOfYear'] / 365)
    df['Hour_sin'] = np.sin(2 * np.pi * df['StartHour'] / 24)
    df['Hour_cos'] = np.cos(2 * np.pi * df['StartHour'] / 24)

    df = df.sort_values(['To', 'Year', 'Month', 'Day', 'StartHour']).reset_index(drop=True)

    for lag in [1, 2, 3, 6, 12, 24]:
        df[f'Lag_{lag}'] = df.groupby('To')['CarCount'].shift(lag)

    df['MA_3'] = df.groupby('To')['CarCount'].transform(lambda x: x.rolling(3).mean())
    df['EMA_5'] = df.groupby('To')['CarCount'].transform(lambda x: x.ewm(span=5, adjust=False).mean())
    df['ROLL12_mean'] = df.groupby('To')['CarCount'].transform(lambda x: x.rolling(12).mean())
    df['ROLL12_std']  = df.groupby('To')['CarCount'].transform(lambda x: x.rolling(12).std())
    df['Diff_1'] = df.groupby('To')['CarCount'].diff(1)

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    feature_cols = [
        'StartHour', 'Hour_sin', 'Hour_cos', 'Day_sin', 'Day_cos',
        'To_encoded', 'MA_3', 'EMA_5', 'ROLL12_mean', 'ROLL12_std', 'Diff_1'
    ] + [f'Lag_{i}' for i in [1, 2, 3, 6, 12, 24]]

    scaler_X = MinMaxScaler()
    df[feature_cols] = scaler_X.fit_transform(df[feature_cols])

    scaler_y = MinMaxScaler()
    df['y_scaled'] = scaler_y.fit_transform(df[['CarCount']])

    df_target = df[df['To'] == TARGET_PLACE].copy()
    if df_target.empty:
        raise ValueError(f"No data found for the target location: {TARGET_PLACE}")
        
    values = df_target[feature_cols + ['y_scaled']].values
    feature_dim = len(feature_cols)

    train_size = int(len(values) * 0.8) 
    test_data = values[train_size:]

    if len(test_data) < INPUT_LEN + 200:
        print(f" Warning: Not enough data for {TARGET_PLACE} to get 200 predictions with an input length of {INPUT_LEN}. Using all available test data.")

    test_dataset  = TrafficDataset(test_data, input_len=INPUT_LEN, feature_dim=feature_dim)
    test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    print(f"Test samples re-created for {TARGET_PLACE}: {len(test_dataset)}")

model_loaded = None
if df is not None:
    try:
        print(f"\nAttempting to load full model from {SAVE_FULL} onto {DEVICE}...")
        model_loaded = torch.load(
            SAVE_FULL, 
            map_location=DEVICE,
            weights_only=False 
        )
        model_loaded.eval()
        print(f" Successfully loaded full model from {SAVE_FULL} onto {DEVICE}.")
        
    except Exception as e:
        print(f"\n Final failure to load full model. Error: {e}")
        model_loaded = None

if model_loaded:
    preds_scaled_all, actuals_scaled_all = [], []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(DEVICE)
            y_pred = model_loaded(X_batch).view(-1).cpu().numpy()
            preds_scaled_all.extend(y_pred)
            actuals_scaled_all.extend(y_batch.view(-1).cpu().numpy())
            
            if len(preds_scaled_all) >= 200:
                break 

    preds_all = scaler_y.inverse_transform(np.array(preds_scaled_all).reshape(-1,1)).flatten()
    actuals_all = scaler_y.inverse_transform(np.array(actuals_scaled_all).reshape(-1,1)).flatten()

    N_SAMPLES = min(200, len(preds_all))
    preds_200 = preds_all[:N_SAMPLES]
    actuals_200 = actuals_all[:N_SAMPLES]

    mae_200 = mean_absolute_error(actuals_200, preds_200)
    mean_predicted_car_count = np.mean(preds_200)
    congestion_level = determine_congestion_level(mean_predicted_car_count, CONGESTION_THRESHOLDS)
 
    print(f"\nPrediction Results for the first {N_SAMPLES} Test Samples at {TARGET_PLACE}:")
    print(f"Mean Absolute Error (MAE) for the subset: {mae_200:.3f}")
    print(f"Mean Predicted Car Count for the subset: {mean_predicted_car_count:.2f}")
    print(f"Inferred Congestion Level: {congestion_level}") 

    #plt.figure(figsize=(12, 6))
    #plt.plot(actuals_200, label='Actual Car Count', linewidth=2)
    #plt.plot(preds_200, label='Predicted Car Count', linestyle='--')
    #plt.title(f'Traffic Forecast for {TARGET_PLACE} (Congestion: {congestion_level})') 
    #plt.xlabel(f'Time Index (Offset from Start of {TARGET_PLACE} Test Set)')
    #plt.ylabel('Car Count')
    #plt.legend()
    #plt.grid(True)
    #plt.tight_layout()
    #plt.show()